In [30]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/made-hw-2/test.csv
/kaggle/input/made-hw-2/sample_submission_most_popular.csv
/kaggle/input/made-hw-2/train.csv


In [31]:
train = pd.read_csv('/kaggle/input/made-hw-2/train.csv')
test = pd.read_csv('/kaggle/input/made-hw-2/test.csv')
sub = pd.read_csv('/kaggle/input/made-hw-2/sample_submission_most_popular.csv')

In [32]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + \
                    ' '.join(emoticons).replace('-', '')
    return text

In [33]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [34]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

clf = OneVsRestClassifier(LogisticRegression())

param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'clf__estimator__penalty': ['l1', 'l2'],
               'clf__estimator__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__estimator__penalty': ['l1', 'l2'],
               'clf__estimator__C': [1.0, 10.0, 100.0]}
             ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', clf)])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                              scoring=make_scorer(f1_score, average='samples'),
                              cv=5,verbose=1,
                              n_jobs=-1)

x_train, y_train = train['dialogue'], train['genres']
x_test = test['dialogue']

gs_lr_tfidf.fit(x_train, y_train)

NameError: name 'f1_score' is not defined

In [ ]:
clf = gs_lr_tfidf.best_estimator_
y_test = clf.predict(x_test)
test.rename(columns={'dialogue': 'genres'})
for w in range(len(y_test)):
    test['genres'] = y_test[w]
compression_opts = dict(method='zip',
                        archive_name='out.csv') 
test.to_csv('out.zip', index=False,
          compression=compression_opts)